In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Copy From Drive to Colab Disk to fetch data faster**

In [2]:
!zip -r /content/drive/MyDrive/DL_Dataset/masks_archive.zip /content/drive/MyDrive/DL_Dataset/masks

Scanning files .


zip error: Interrupted (aborting)


In [11]:
# Create the local destination folder
!mkdir -p /content/masks

# Copy the single zip file (very fast)
!cp /content/drive/MyDrive/DL_Dataset/masks_archive.zip /content/masks_archive.zip

# Unzip locally (extremely fast)
!unzip -q /content/masks_archive.zip -d /content/masks

# **Environment Setup & Data Extraction**

In [89]:
!pip install -U bitsandbytes qwen-vl-utils accelerate natsort

# Extract the Data

In [3]:
import zipfile, os, shutil
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from natsort import natsorted

# --- CONFIGURATION ---
zip_path = '/content/masks_.zip'
extract_path = '/content/DL_Masks'

# 1. Clean and Extract
if os.path.exists(extract_path): shutil.rmtree(extract_path)
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zf:
    files = zf.namelist()
    for file in tqdm(files, desc="Unzipping Dataset"):
        zf.extract(file, extract_path)

# 2. Dynamic Search (Fixes the KeyError)
search_root = Path(extract_path)
all_records = []

# Find all image files
extensions = ['*.jpg', '*.JPG', '*.jpeg', '*.png', '*.bmp']
image_files = []
for ext in extensions:
    image_files.extend(list(search_root.rglob(ext)))

for img_p in image_files:
    # Get the parent folder (e.g., 'Mayo 3')
    mayo_folder = img_p.parent
    # Get the grandparent folder (e.g., '101')
    patient_folder = mayo_folder.parent

    # Validation: Ensure we are actually looking at a Patient ID (numbered folder)
    # and a Mayo class folder
    if "mayo" in mayo_folder.name.lower():
        all_records.append({
            "patient": str(patient_folder.name), # This will be '1', '10', etc.
            "mayo_class": str(mayo_folder.name), # This will be 'Mayo 0', etc.
            "img_path": str(img_p)
        })

# 3. Save and Verify
if not all_records:
    print("\n❌ ERROR: Still found 0 images. Check if files are deeply nested or have different extensions.")
else:
    df_index = pd.DataFrame(all_records)
    df_index.to_csv("/content/drive/MyDrive/DL_Dataset/master_dataset_index_original.csv", index=False)

    print(f"\n✅ SUCCESS: Found {len(df_index)} images.")
    print(f"📊 Patient Count: {df_index['patient'].nunique()}")
    print(f"📁 Classes detected: {df_index['mayo_class'].unique()[:4]}")

    # Print sample to be 100% sure
    print("\nSample mapping:")
    print(df_index[['patient', 'mayo_class']].head(3))

print("\n✅ Done")

Unzipping Dataset: 100%|██████████| 7471/7471 [00:00<00:00, 10928.16it/s]



✅ SUCCESS: Found 6395 images.
📊 Patient Count: 312
📁 Classes detected: ['Mayo_1' 'Mayo_0' 'Mayo_3' 'Mayo_2']

Sample mapping:
  patient mayo_class
0     367     Mayo_1
1     367     Mayo_1
2     367     Mayo_1

✅ Done


In [4]:
df = pd.read_csv("/content/enhanced_7B_descriptive.csv")

In [5]:
import os

# --- STEP 1: Corrected Roots ---
# Based on your 'ls' output, these are the confirmed local roots
local_img_root = "/content/DL_Dataset/patient_based_classified_images"
local_mask_root = "/content/DL_Masks/masks_"

def fix_img_path(old_path):
    if 'patient_based_classified_images' in str(old_path):
        # 1. Extract the relative part (e.g., '1/Mayo 0/UC_patient_1_16.png')
        relative_part = old_path.split('patient_based_classified_images/')[-1]

        # 2. Change extension from .png to .bmp if needed
        if relative_part.endswith('.png'):
            relative_part = relative_part.replace('.png', '.bmp')

        # 3. Combine with new local root
        return os.path.join(local_img_root, relative_part)
    return old_path

def fix_mask_path(old_path):
    if 'masks' in str(old_path):
        relative_part = old_path.split('masks/')[-1]
        return os.path.join(local_mask_root, relative_part)
    return old_path

# Apply the fixes to the dataframe
df['original_image'] = df['original_image'].apply(fix_img_path)
df['mask_image'] = df['mask_image'].apply(fix_mask_path)

# --- STEP 2: Final Verification ---
img_check = df['original_image'].iloc[0]
mask_check = df['mask_image'].iloc[0]

print(f"New Image Path: {img_check}")
print(f"Image Exists? {'✅' if os.path.exists(img_check) else '❌'}")

print(f"\nNew Mask Path: {mask_check}")
print(f"Mask Exists?  {'✅' if os.path.exists(mask_check) else '❌'}")

New Image Path: /content/DL_Dataset/patient_based_classified_images/1/Mayo 0/UC_patient_1_16.bmp
Image Exists? ✅

New Mask Path: /content/DL_Masks/masks_/1/Mayo_0/mask_UC_patient_1_16.png
Mask Exists?  ✅


In [6]:
# Save the updated DataFrame locally
df.to_csv('/content/enhanced_7B_descriptive.csv', index=False)

print("✅ Updated CSV saved to /content/enhanced_7B_descriptive.csv")

✅ Updated CSV saved to /content/enhanced_7B_descriptive.csv


# **Configuration**

In [ ]:
import os
import cv2
import glob
import numpy as np
import torch
from pathlib import Path

class Config:
    # Processing Thresholds
    DARK_THRESHOLD = 60
    BRIGHT_THRESHOLD = 200
    CLAHE_CLIP_LIMIT = 2.0
    CLAHE_GRID_SIZE = (8, 8)

    # Paths
    INPUT_DIR = "/content/drive/MyDrive/DL_Dataset"
    PROCESSED_DIR = "/content/drive/MyDrive/Preprocessed_Dataset"

    # Sampling
    MAX_IMAGES = 11280

    # Model (Optimized for 80GB VRAM)
    MODEL_ID = "natong19/Qwen2-VL-7B-Instruct-abliterated"

# Initialize directories
os.makedirs(Config.PROCESSED_DIR, exist_ok=True)

print(f"✅ Configuration set for {Config.MODEL_ID}")
print(f"📂 Output directory: {Config.PROCESSED_DIR}")

✅ Configuration set for natong19/Qwen2-VL-7B-Instruct-abliterated
📂 Output directory: /content/drive/MyDrive/Preprocessed_Dataset


# **Preprocessing Pipeline**

In [ ]:
class MedicalImagePreprocessor:
    """
    Comprehensive image preprocessing for colonoscopy images.
    Handles:
    - Black border removal
    - Dark image enhancement (CLAHE)
    - Bright image normalization
    - Contrast adjustment
    """

    def __init__(self,
                 dark_threshold=60,
                 bright_threshold=200,
                 clahe_clip_limit=2.0,
                 clahe_grid_size=(8, 8)):
        """
        Initialize preprocessor with thresholds.

        Args:
            dark_threshold: Mean brightness below this triggers CLAHE
            bright_threshold: Mean brightness above this triggers dimming
            clahe_clip_limit: CLAHE contrast limit
            clahe_grid_size: CLAHE tile grid size
        """
        self.dark_threshold = dark_threshold
        self.bright_threshold = bright_threshold
        self.clahe_clip_limit = clahe_clip_limit
        self.clahe_grid_size = clahe_grid_size

        # Create CLAHE object
        self.clahe = cv2.createCLAHE(
            clipLimit=clahe_clip_limit,
            tileGridSize=clahe_grid_size
        )

    def get_image_stats(self, image):
        """
        Calculate image brightness and contrast statistics.

        Args:
            image: BGR or RGB image (numpy array)

        Returns:
            dict with mean_brightness, std_brightness, is_dark, is_bright
        """
        # Convert to grayscale for analysis
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image

        mean_brightness = np.mean(gray)
        std_brightness = np.std(gray)

        return {
            'mean_brightness': mean_brightness,
            'std_brightness': std_brightness,
            'is_dark': mean_brightness < self.dark_threshold,
            'is_bright': mean_brightness > self.bright_threshold,
            'is_low_contrast': std_brightness < 30
        }

    def remove_black_borders(self, image, threshold=10):
        """
        Remove black borders common in endoscopy images.

        Args:
            image: BGR image
            threshold: Pixel value threshold for black detection

        Returns:
            Cropped image with borders removed
        """
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Find non-black regions
        _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            # Get bounding box of largest contour
            largest_contour = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(largest_contour)

            # Add small padding
            padding = 5
            x = max(0, x - padding)
            y = max(0, y - padding)
            w = min(image.shape[1] - x, w + 2 * padding)
            h = min(image.shape[0] - y, h + 2 * padding)

            # Crop
            cropped = image[y:y+h, x:x+w]

            # Only return cropped if it's a reasonable size
            if cropped.shape[0] > 50 and cropped.shape[1] > 50:
                return cropped

        return image

    def apply_clahe(self, image):
        """
        Apply CLAHE (Contrast Limited Adaptive Histogram Equalization).
        Enhances contrast in dark images while preventing over-amplification.

        Args:
            image: BGR image

        Returns:
            CLAHE enhanced image
        """
        # Convert to LAB color space
        lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

        # Split channels
        l, a, b = cv2.split(lab)

        # Apply CLAHE to L channel (lightness)
        l_clahe = self.clahe.apply(l)

        # Merge channels
        lab_clahe = cv2.merge([l_clahe, a, b])

        # Convert back to BGR
        result = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

        return result

    def normalize_brightness(self, image, target_brightness=128):
        """
        Normalize overly bright images.

        Args:
            image: BGR image
            target_brightness: Target mean brightness

        Returns:
            Brightness-normalized image
        """
        # Convert to HSV
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)

        # Calculate current mean brightness
        current_brightness = np.mean(v)

        # Calculate adjustment factor
        if current_brightness > 0:
            factor = target_brightness / current_brightness
            factor = np.clip(factor, 0.5, 1.5)  # Limit adjustment

            # Apply adjustment
            v = np.clip(v * factor, 0, 255).astype(np.uint8)

        # Merge and convert back
        hsv_adjusted = cv2.merge([h, s, v])
        result = cv2.cvtColor(hsv_adjusted, cv2.COLOR_HSV2BGR)

        return result

    def adjust_gamma(self, image, gamma=1.0):
        """
        Apply gamma correction.
        gamma < 1: Brighten image
        gamma > 1: Darken image

        Args:
            image: BGR image
            gamma: Gamma value

        Returns:
            Gamma-corrected image
        """
        inv_gamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** inv_gamma) * 255
                          for i in np.arange(0, 256)]).astype(np.uint8)
        return cv2.LUT(image, table)

    def enhance_contrast(self, image, factor=1.2):
        """
        Enhance contrast using simple stretching.

        Args:
            image: BGR image
            factor: Contrast enhancement factor

        Returns:
            Contrast-enhanced image
        """
        # Convert to float
        img_float = image.astype(np.float32)

        # Apply contrast
        mean = np.mean(img_float, axis=(0, 1), keepdims=True)
        img_contrast = (img_float - mean) * factor + mean

        # Clip and convert back
        result = np.clip(img_contrast, 0, 255).astype(np.uint8)

        return result

    def preprocess(self, image_path, save_path=None):
        """
        Full preprocessing pipeline.

        Args:
            image_path: Path to input image
            save_path: Optional path to save processed image

        Returns:
            processed_image: Preprocessed BGR image
            processing_info: Dict with processing details
        """
        # Read image
        image = cv2.imread(str(image_path))
        if image is None:
            raise ValueError(f"Could not read image: {image_path}")

        original = image.copy()
        processing_steps = []

        # Step 1: Get initial stats
        initial_stats = self.get_image_stats(image)

        # Step 2: Remove black borders
        image = self.remove_black_borders(image)
        if image.shape != original.shape:
            processing_steps.append("border_removal")

        # Step 3: Get stats after border removal
        stats = self.get_image_stats(image)

        # Step 4: Apply appropriate enhancement based on brightness
        if stats['is_dark']:
            # Dark image: Apply CLAHE + slight gamma correction
            image = self.apply_clahe(image)
            image = self.adjust_gamma(image, gamma=0.85)  # Brighten slightly
            processing_steps.append(f"clahe_dark (brightness: {stats['mean_brightness']:.1f})")

        elif stats['is_bright']:
            # Bright image: Normalize brightness + slight contrast boost
            image = self.normalize_brightness(image, target_brightness=140)
            image = self.adjust_gamma(image, gamma=1.15)  # Darken slightly
            processing_steps.append(f"normalize_bright (brightness: {stats['mean_brightness']:.1f})")

        elif stats['is_low_contrast']:
            # Low contrast: Apply mild CLAHE
            temp_clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8, 8))
            lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
            l, a, b = cv2.split(lab)
            l = temp_clahe.apply(l)
            lab = cv2.merge([l, a, b])
            image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
            processing_steps.append(f"mild_clahe_contrast (std: {stats['std_brightness']:.1f})")

        # Step 5: Final stats
        final_stats = self.get_image_stats(image)

        # Save if path provided
        if save_path:
            cv2.imwrite(str(save_path), image)

        processing_info = {
            'initial_brightness': initial_stats['mean_brightness'],
            'initial_contrast': initial_stats['std_brightness'],
            'final_brightness': final_stats['mean_brightness'],
            'final_contrast': final_stats['std_brightness'],
            'was_dark': initial_stats['is_dark'],
            'was_bright': initial_stats['is_bright'],
            'was_low_contrast': initial_stats['is_low_contrast'],
            'processing_steps': processing_steps,
            'processed': len(processing_steps) > 0
        }

        return image, processing_info

    def preprocess_for_vlm(self, image_path, temp_dir=None):
        """
        Preprocess image and save to temp location for VLM.

        Args:
            image_path: Original image path
            temp_dir: Directory to save processed image

        Returns:
            processed_path: Path to processed image
            processing_info: Processing details
        """
        if temp_dir is None:
            temp_dir = Config.PROCESSED_DIR

        # Generate output path
        filename = os.path.basename(image_path)
        base, ext = os.path.splitext(filename)
        processed_path = os.path.join(temp_dir, f"{base}_processed.png")

        # Preprocess and save
        processed_image, processing_info = self.preprocess(image_path, processed_path)
        processing_info['processed_path'] = processed_path

        return processed_path, processing_info

    def run_pipeline(self, image_path, input_root, output_root):
        """
        Preprocesses an image and saves it to a mirrored directory structure.

        Args:
            image_path (str): Path to the original input image.
            input_root (str): The root directory of the input dataset.
            output_root (str): The root directory for the preprocessed output.

        Returns:
            str: Path to the saved preprocessed image, or None if an error occurs.
        """
        try:
            relative_path = os.path.relpath(image_path, input_root)
            output_dir = os.path.join(output_root, os.path.dirname(relative_path))
            os.makedirs(output_dir, exist_ok=True)

            # Ensure the output filename has a consistent extension, e.g., .png
            base_name, _ = os.path.splitext(os.path.basename(image_path))
            save_path = os.path.join(output_dir, f"{base_name}.png")

            self.preprocess(image_path, save_path)
            return save_path
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            return None


# Create global preprocessor instance
preprocessor = MedicalImagePreprocessor(
    dark_threshold=Config.DARK_THRESHOLD,
    bright_threshold=Config.BRIGHT_THRESHOLD,
    clahe_clip_limit=Config.CLAHE_CLIP_LIMIT,
    clahe_grid_size=Config.CLAHE_GRID_SIZE
)

print("✅ MedicalImagePreprocessor defined!")
print(f"   Dark threshold: {Config.DARK_THRESHOLD}")
print(f"   Bright threshold: {Config.BRIGHT_THRESHOLD}")
print(f"   CLAHE clip limit: {Config.CLAHE_CLIP_LIMIT}")

✅ MedicalImagePreprocessor defined!
   Dark threshold: 60
   Bright threshold: 200
   CLAHE clip limit: 2.0


In [ ]:
# --- Configuration Parameters ---
# Ensure these are defined in your Config class or passed directly
preprocessor = MedicalImagePreprocessor(
    dark_threshold=Config.DARK_THRESHOLD,
    bright_threshold=Config.BRIGHT_THRESHOLD,
    clahe_clip_limit=Config.CLAHE_CLIP_LIMIT,
    clahe_grid_size=Config.CLAHE_GRID_SIZE
)

# Fetch all images recursively from all subfolders
image_extensions = ('*.bmp', '*.jpg', '*.jpeg', '*.png', '*.BMP', '*.JPG')
raw_images = []
for ext in image_extensions:
    raw_images.extend(glob.glob(f"{Config.INPUT_DIR}/**/{ext}", recursive=True))

# Sample for testing or process all
sampled_images = raw_images[:Config.MAX_IMAGES]

processed_list = []
print(f"🚀 Mirroring structure: {Config.INPUT_DIR} -> {Config.PROCESSED_DIR}")
print(f"📦 Processing {len(sampled_images)} images...")

for img_path in sampled_images:
    # run_pipeline now handles both enhancement and folder creation
    out_path = preprocessor.run_pipeline(img_path, Config.INPUT_DIR, Config.PROCESSED_DIR)

    if out_path:
        processed_list.append(out_path)
        # Print the relative path to confirm structure is maintained
        display_rel = os.path.relpath(out_path, Config.PROCESSED_DIR)
        print(f"✅ Created: {display_rel}")

print(f"\n✨ Successfully mirrored {len(processed_list)} images with clinical enhancements.")

Streaming output truncated to the last 5000 lines.
✅ Created: patient_based_classified_images/386/Mayo 0/UC_patient_386_79.png
✅ Created: patient_based_classified_images/386/Mayo 0/UC_patient_386_80.png
✅ Created: patient_based_classified_images/386/Mayo 0/UC_patient_386_81.png
✅ Created: patient_based_classified_images/386/Mayo 0/UC_patient_386_82.png
✅ Created: patient_based_classified_images/386/Mayo 0/UC_patient_386_83.png
✅ Created: patient_based_classified_images/386/Mayo 1/UC_patient_386_1.png
✅ Created: patient_based_classified_images/386/Mayo 1/UC_patient_386_13.png
✅ Created: patient_based_classified_images/386/Mayo 1/UC_patient_386_14.png
✅ Created: patient_based_classified_images/386/Mayo 1/UC_patient_386_16.png
✅ Created: patient_based_classified_images/386/Mayo 1/UC_patient_386_17.png
✅ Created: patient_based_classified_images/386/Mayo 1/UC_patient_386_18.png
✅ Created: patient_based_classified_images/386/Mayo 1/UC_patient_386_19.png
✅ Created: patient_based_classified_im

# **Preprocesssing Folder Mapping and Sorting**

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from natsort import natsorted

# --- CONFIGURATION ---
# Points to the directory where your mirrored preprocessed files are stored
preprocessed_path = '/content/drive/MyDrive/Preprocessed_Dataset/patient_based_classified_images'
search_root = Path(preprocessed_path)

if not search_root.exists():
    print(f"❌ ERROR: Preprocessed folder not found at {preprocessed_path}")
else:
    all_preprocessed_records = []

    # 1. Dynamic Search for Preprocessed Images
    extensions = ['*.jpg', '*.JPG', '*.jpeg', '*.png', '*.bmp', '*.BMP']
    image_files = []
    for ext in extensions:
        image_files.extend(list(search_root.rglob(ext)))

    print(f"🔍 Scanning {len(image_files)} preprocessed files...")

    # 2. Extract Patient and Class from Mirrored Structure
    for img_p in tqdm(image_files, desc="Indexing Preprocessed Files"):
        # Structure is mirrored: root/PatientID/MayoClass/Image.ext
        # img_p.parent is 'Mayo X'
        mayo_folder = img_p.parent
        # img_p.parent.parent is 'Patient ID'
        patient_folder = mayo_folder.parent

        # Validate that we are looking at the correct depth (mirrored folders)
        if "mayo" in mayo_folder.name.lower():
            all_preprocessed_records.append({
                "patient": str(patient_folder.name),
                "mayo_class": str(mayo_folder.name),
                "img_path": str(img_p),
                "is_preprocessed": True
            })

    # 3. Save and Verify
    if not all_preprocessed_records:
        print("\n❌ ERROR: Found 0 images in the preprocessed folder.")
    else:
        df_preprocessed = pd.DataFrame(all_preprocessed_records)
        # Save a specific index for preprocessed files
        index_save_path = "/content/drive/MyDrive/Preprocessed_Dataset/preprocessed_dataset_index.csv"
        df_preprocessed.to_csv(index_save_path, index=False)

        print(f"\n✅ SUCCESS: Indexed {len(df_preprocessed)} preprocessed images.")
        print(f"📊 Patient Count: {df_preprocessed['patient'].nunique()}")
        print(f"📁 Classes detected: {df_preprocessed['mayo_class'].unique()}")
        print(f"💾 Index saved to: {index_save_path}")

        # Display sample mapping
        print("\nSample mapping (Preprocessed):")
        print(df_preprocessed[['patient', 'mayo_class', 'img_path']].head(3))

🔍 Scanning 11276 preprocessed files...


Indexing Preprocessed Files: 100%|██████████| 11276/11276 [00:00<00:00, 114867.50it/s]


✅ SUCCESS: Indexed 11276 preprocessed images.


📊 Patient Count: 564
📁 Classes detected: ['Mayo 0' 'Mayo 1' 'Mayo 2' 'Mayo 3']
💾 Index saved to: /content/drive/MyDrive/Preprocessed_Dataset/preprocessed_dataset_index.csv

Sample mapping (Preprocessed):
  patient mayo_class                                           img_path
0       1     Mayo 0  /content/drive/MyDrive/Preprocessed_Dataset/pa...
1       1     Mayo 0  /content/drive/MyDrive/Preprocessed_Dataset/pa...
2       1     Mayo 0  /content/drive/MyDrive/Preprocessed_Dataset/pa...


# **QWEN VL 7B Instruct + MedSam**

### **Dependencies and Initialization**

In [41]:
import zipfile, os, shutil
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from natsort import natsorted
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq, SamModel, SamProcessor, BitsAndBytesConfig
import cv2
import numpy as np
import matplotlib.pyplot as plt
import textwrap
import gc
import PIL.Image
import re
from qwen_vl_utils import process_vision_info

In [ ]:
# Configuration for 4-bit loading
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

# SWITCH TO THE ABLITERATED VERSION HERE
model_name = "natong19/Qwen2-VL-7B-Instruct-abliterated"

model_qwen = AutoModelForVision2Seq.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto", # Automatically balances memory
    trust_remote_code=True,
    _attn_implementation="sdpa"
)
processor_qwen = AutoProcessor.from_pretrained(model_name)

# MedSAM remains the same as it is the best for medical segmentation
medsam_model_id = "flaviagiammarino/medsam-vit-base"
medsam_model = SamModel.from_pretrained(medsam_model_id).to("cuda")
medsam_processor = SamProcessor.from_pretrained(medsam_model_id)

print("🚀 Qwen2-VL-7B (4-bit) & MedSAM Loaded. Ready for robust colonoscopy analysis.")

### **Core Inference Functions**

In [ ]:
import PIL.Image
import re
import numpy as np
from qwen_vl_utils import process_vision_info

# Keep MedSAM as it is
def get_medsam_mask(image_path, norm_box):
    """Refines box with strict safety clamping to prevent CUDA crashes."""
    image = PIL.Image.open(image_path).convert("RGB")
    w, h = image.size

    safe_box = [max(0, min(1000, int(x))) for x in norm_box]
    ymin, xmin, ymax, xmax = safe_box

    if ymax <= ymin: ymax = min(1000, ymin + 50)
    if xmax <= xmin: xmax = min(1000, xmin + 50)

    input_box = [xmin * w / 1000, ymin * h / 1000, xmax * w / 1000, ymax * h / 1000]

    inputs = medsam_processor(image, input_boxes=[[input_box]], return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = medsam_model(**inputs)

    masks = medsam_processor.image_processor.post_process_masks(
        outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu()
    )
    return masks[0][0][0].numpy() > 0, np.array(input_box)

def run_qwen_clinical_7B(img_p, m_class):
    """
    72B Inference focused on grounding and detailed clinical description.
    - Uses 'm_class' only to provide professional terminology steerage.
    - Removes explicit classification to prioritize caption fidelity.
    """

    # Clinical steerage provides the model with professional 'vocabulary'
    hints = {
        "Mayo 0": "normal colonic mucosa with clear, delicate branching vascularity and a smooth surface.",
        "Mayo 1": "mild erythema and attenuated or decreased vascular patterns; focus on subtle loss of vessel detail.",
        "Mayo 2": "marked erythema, obliterated vascularity, mucosal friability, and granular textures or erosions.",
        "Mayo 3": "spontaneous bleeding and deep, well-defined ulcerations."
    }
    context_hint = hints.get(m_class, "colonic mucosal assessment")

    messages = [{"role": "user", "content": [
        {"type": "image", "image": f"file://{img_p}"},
        {"type": "text", "text": (
            "ANATOMIC CONTEXT: High-resolution COLONOSCOPY image. Subject: COLONIC TISSUE.\n"
            "ROLE: Senior GI Specialist performing a technical mucosal audit.\n\n"
            f"CLINICAL CONTEXT: Observations should be consistent with {context_hint}.\n\n"
            "TASK:\n"
            "1. Analyze the mucosal surface architecture and vascular patterns visible in this specific frame.\n"
            "2. Localize the area of highest clinical interest and provide its grounding coordinates.\n"
            "3. Provide a 'DESCRIPTION:' with a technical assessment of the mucosal surface and vascular architecture.\n\n"
            "OUTPUT FORMAT:\n"
            "1. Bounding Box: [ymin, xmin, ymax, xmax] (normalized 0-1000).\n"
            "2. DESCRIPTION: Provide a natural, technical analysis using professional terminology "
            "(e.g., 'tortuous vessels', 'fibrinopurulent exudate', 'friable mucosa').\n\n"
            "CONSTRAINTS:\n"
            "- Artifact Awareness: Internally distinguish light reflections from tissue pathology. Do not describe artifacts.\n"
            "- Self-Correction: Verify anatomy as COLON. Do not reference other organs.\n"
            "- Do not use numerical grades (0-3) within the description text block."
        )}
    ]}]

    # Apply template and process vision tokens
    text = processor_qwen.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    img_in, _ = process_vision_info(messages)

    # Optimization: Use bfloat16 for input tensors to match model dtype for 80GB VRAM stability
    inputs = processor_qwen(text=[text], images=img_in, return_tensors="pt").to("cuda").to(torch.bfloat16)

    with torch.no_grad():
        out_ids = model_qwen.generate(
            **inputs,
            max_new_tokens=600,
            temperature=0.2,
            do_sample=True,
            use_cache=True
        )

    # Decode assistant response correctly by slicing the input tokens
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, out_ids)]
    raw_res = processor_qwen.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    # --- EXTRACTION ---
    box_match = re.search(r'\[(\d+),\s*(\d+),\s*(\d+),\s*(\d+)\]', raw_res)
    box = [int(x) for x in box_match.groups()] if box_match else [200, 200, 800, 800]

    # Extract only the findings text
    clinical_text = raw_res.split("DESCRIPTION:")[-1].strip() if "DESCRIPTION:" in raw_res else raw_res

    return {"box": box, "caption": clinical_text}

print("🚀 Descriptive Inference Function Ready.")

🚀 Descriptive Inference Function Ready.


# **Unified Generation Loop (Pilot)**

In [ ]:
# --- DIRECTORY SETUP ---
dataset_root = "/content/drive/MyDrive/DL_Dataset"
output_csv = f"{dataset_root}/enhanced_7B_descriptive.csv"
os.makedirs(f"{dataset_root}/masks", exist_ok=True)

# --- LOAD PROGRESS ---
if os.path.exists(output_csv):
    df_progress = pd.read_csv(output_csv)
    processed_paths = set(df_progress['original_image'].tolist())
    print(f"🔄 Resuming: {len(processed_paths)} images already processed.")
else:
    processed_paths = set()
    print("🚀 Starting fresh descriptive generation.")

df_index = pd.read_csv("/content/drive/MyDrive/Preprocessed_Dataset/preprocessed_dataset_index.csv")
patients = natsorted(df_index['patient'].unique())[:564]

print(f"📊 Target Patients: {patients}\n")

for p_id in patients:
    patient_data = df_index[df_index['patient'].astype(str) == str(p_id)]
    eval_count = 0

    for idx, row in patient_data.iterrows():
        img_p, m_label = row['img_path'], row['mayo_class']

        # --- SKIP CHECK ---
        if img_p in processed_paths:
            continue

        m_folder = m_label.replace(' ', '_')
        save_dir = f"{dataset_root}/masks/{p_id}/{m_folder}"
        os.makedirs(save_dir, exist_ok=True)

        try:
            # Step 1: Descriptive 72B Inference
            # Using the run_qwen_clinical_descriptive function defined previously
            q_res = run_qwen_clinical_7B(img_p, m_label)

            if not isinstance(q_res['box'], list) or len(q_res['box']) != 4:
                continue

            # Step 2: MedSAM Segmentation
            mask, s_box = get_medsam_mask(img_p, q_res['box'])

            # Step 3: Save Mask (Immediate I/O)
            mask_filename = f"mask_{os.path.basename(img_p)}"
            mask_save_path = os.path.join(save_dir, mask_filename)
            cv2.imwrite(mask_save_path, mask.astype(np.uint8) * 255)

            # Step 4: Visual Evaluation (First 2 of each patient)
            if eval_count < 1:
                fig, ax = plt.subplots(1, 2, figsize=(14, 6))
                img_rgb = cv2.cvtColor(cv2.imread(img_p), cv2.COLOR_BGR2RGB)

                ax[0].imshow(img_rgb)
                ax[0].set_title(f"P{p_id} | Context: {m_label}", fontsize=10)
                ax[0].axis('off')

                ax[1].imshow(img_rgb)
                overlay = np.zeros_like(img_rgb)
                overlay[mask] = [0, 180, 255] # Clinical blue
                ax[1].imshow(overlay, alpha=0.4)

                # Wrapped Caption Display
                ax[1].set_title("\n".join(textwrap.wrap(q_res['caption'], 60)),
                                fontsize=9, color='darkblue', loc='center', style='italic')
                ax[1].axis('off')
                plt.show(); plt.close(fig)
                eval_count += 1

            # Step 5: Metadata Record (Removed Prediction Column)
            new_data = {
                "patient_id": p_id,
                "original_image": img_p,
                "mask_image": mask_save_path,
                "clinical_caption": q_res['caption'],
                "original_mayo": m_label,
                "bbox_pixels": s_box.tolist(),
                "caption_length": len(q_res['caption'])
            }

            # --- INCREMENTAL SAVE ---
            pd.DataFrame([new_data]).to_csv(output_csv, mode='a', index=False, header=not os.path.exists(output_csv))
            processed_paths.add(img_p)

            print(f"✅ Saved P{p_id} | {os.path.basename(img_p)} | Caption Length: {len(q_res['caption'])}")

        except Exception as e:
            print(f"❌ Error on {img_p}: {e}")

        torch.cuda.empty_cache()
        gc.collect()

# Final Summary Generation (Replaces Classification Report)
if os.path.exists(output_csv):
    df_final = pd.read_csv(output_csv)
    print("\n" + "="*40)
    print("📝 GENERATION SUMMARY REPORT")
    print("="*40)
    print(f"Total Images Processed: {len(df_final)}")
    print(f"Average Caption Length: {df_final['caption_length'].mean():.1f} characters")
    print("\nMean Detail Level by Class:")
    print(df_final.groupby('original_mayo')['caption_length'].mean())

### **Final Metadata Verification**

In [ ]:
# Check the generated CSV
final_df = pd.read_csv(f"{dataset_root}/enhanced_metadata.csv")
print("\n🔥 Final CSV Verification:")
display(final_df[['clinical_caption', 'bbox_pixels', 'mayo_label']].head())


🔥 Final CSV Verification:


,clinical_caption,bbox_pixels,mayo_label
0,The most significant inflammatory area is loca...,"[70.4, 28.8, 140.8, 86.4]",Mayo 2
1,The image shows a marked area of erythema with...,"[70.4, 57.6, 281.6, 230.4]",Mayo 2
2,**\nThe image depicts a colonoscopy view of th...,"[70.4, 57.6, 281.6, 230.4]",Mayo 2
3,The image shows a region of mild erythema with...,"[70.4, 28.8, 140.8, 86.4]",Mayo 1
4,The colon shows mild erythema with decreased v...,"[70.4, 57.6, 281.6, 230.4]",Mayo 1


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import textwrap
import cv2
import os

print("="*60)
print("🚀 GENERATING CLINICAL EVALUATION DASHBOARDS")
print("="*60)

def display_detailed_7B_result(row, idx, total):
    """
    Adapts the professional 4-panel dashboard to the 72B + MedSAM workflow.
    Panels: Original -> Mask Overlay -> Mask Binary -> Technical Findings
    """
    # 1. Load Original Image
    raw_img = cv2.imread(row['original_image'])
    if raw_img is None: return None
    img_original = cv2.cvtColor(raw_img, cv2.COLOR_BGR2RGB)

    # 2. Load Mask
    mask_img = cv2.imread(row['mask_image'], cv2.IMREAD_GRAYSCALE)
    mask = mask_img > 0 if mask_img is not None else None

    # Create figure
    fig = plt.figure(figsize=(24, 10))

    # Status Logic (Comparing 72B Prediction vs Ground Truth)
    # Predicted Mayo comes from run_qwen_clinical_72B_V3
    pred_label = row['predicted_mayo']
    gt_label = row['original_mayo']
    is_correct = pred_label.strip().lower() == gt_label.strip().lower()

    status_color = '#2ecc71' if is_correct else '#e74c3c'
    status_text = "✅ CLINICAL MATCH" if is_correct else "❌ MISMATCH"

    fig.suptitle(
        f"72B EVALUATION {idx+1}/{total} | Patient: {row['patient_id']} | "
        f"GT: {gt_label} | PRED: {pred_label} | {status_text}",
        fontsize=18, fontweight='bold', color=status_color, y=1.02
    )

    gs = fig.add_gridspec(2, 4, height_ratios=[3, 1.2], hspace=0.3, wspace=0.2)

    # --- PANEL 1: ORIGINAL ---
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.imshow(img_original)
    ax1.set_title("1. INPUT SCAN", fontsize=14, fontweight='bold')
    ax1.axis('off')

    # --- PANEL 2: SEGMENTATION OVERLAY ---
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.imshow(img_original)
    if mask is not None:
        overlay = np.zeros((*img_original.shape[:2], 4))
        overlay[mask, :] = [0, 0.7, 1.0, 0.45]  # Clinical Blue
        ax2.imshow(overlay)

        # Draw the Qwen-72B Grounding Box
        box = row['bbox_pixels'] # [xmin, ymin, xmax, ymax]
        rect = Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1],
                         linewidth=2, edgecolor='#f1c40f', facecolor='none', linestyle='--')
        ax2.add_patch(rect)

    ax2.set_title("2. REGION OF INTEREST", fontsize=14, fontweight='bold')
    ax2.axis('off')

    # --- PANEL 3: BINARY MASK ---
    ax3 = fig.add_subplot(gs[0, 2])
    if mask is not None:
        mask_viz = np.zeros((*mask.shape, 3), dtype=np.uint8)
        mask_viz[mask] = [0, 180, 255]
        mask_viz[~mask] = [30, 30, 30]
        ax3.imshow(mask_viz)
    ax3.set_title("3. MEDSAM EXTRACTION", fontsize=14, fontweight='bold')
    ax3.axis('off')

    # --- PANEL 4: ANALYTICS ---
    ax4 = fig.add_subplot(gs[0, 3])
    # Calculate area coverage
    area_pct = (np.sum(mask) / (mask.shape[0] * mask.shape[1])) * 100 if mask is not None else 0

    ax4.text(0.1, 0.8, f"SPATIAL DATA", fontsize=12, fontweight='bold', color='purple')
    ax4.text(0.1, 0.7, f"• Norm Box: {row['bbox_normalized']}", fontsize=10)
    ax4.text(0.1, 0.6, f"• Pixel Box: {str(row['bbox_pixels'])}", fontsize=10)
    ax4.text(0.1, 0.5, f"• Coverage: {area_pct:.2f}%", fontsize=10)

    ax4.text(0.1, 0.3, f"MODEL METADATA", fontsize=12, fontweight='bold', color='blue')
    ax4.text(0.1, 0.2, f"• Architecture: Qwen2-VL-72B", fontsize=10)
    ax4.text(0.1, 0.1, f"• Precision: 4-bit NF4", fontsize=10)
    ax4.axis('off')

    # --- BOTTOM PANEL: CLINICAL FINDINGS ---
    ax_info = fig.add_subplot(gs[1, :])
    ax_info.axis('off')

    findings = row['clinical_caption']

    info_table = f"""
┌───────────────────────────────────────┬───────────────────────────────────────┬───────────────────────────────────────────┐
│  Ground Truth: {gt_label:<22} │  Predicted: {pred_label:<25} │  Match Status: {status_text:<26} │
└───────────────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────────┘
"""

    # Findings Box
    ax_info.text(0, 0.95, info_table, transform=ax_info.transAxes, fontfamily='monospace', fontsize=11, verticalalignment='top')

    wrapped_findings = "\n".join(textwrap.wrap(f"TECHNICAL DESCRIPTION: {findings}", 155))
    ax_info.text(0, 0.6, wrapped_findings, transform=ax_info.transAxes,
                 fontsize=12, fontfamily='serif', verticalalignment='top',
                 bbox=dict(boxstyle='round,pad=1', facecolor='#fdfefe', edgecolor='#bdc3c7', alpha=1))

    plt.tight_layout()

    # Save to the local directory for easy download
    save_fn = f"eval_P{row['patient_id']}_{os.path.basename(row['original_image'])}"
    plt.savefig(f"{dataset_root}/{save_fn}", dpi=120, bbox_inches='tight')
    plt.show()
    plt.close()

# Convert results to DataFrame if not already
results_df = pd.DataFrame(processed_results)

for idx, row in results_df.iterrows():
    display_detailed_7B_result(row, idx, len(results_df))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import textwrap
import os

def run_final_presentation_eval(csv_path, num_samples=7):
    """Evaluates the hierarchical dataset with strict text-mapping fixes."""
    if not os.path.exists(csv_path):
        print("❌ Metadata file not found.")
        return

    df = pd.read_csv(csv_path)
    # Use a fixed seed for presentation consistency
    samples = df.sample(n=num_samples, random_state=42)

    for _, row in samples.iterrows():
        # Load images
        img = cv2.cvtColor(cv2.imread(row['original_image']), cv2.COLOR_BGR2RGB)
        mask_path = row['mask_image']

        # Create professional layout
        fig, axes = plt.subplots(1, 2, figsize=(8, 4), facecolor='white')

        # 1. Left: Original Image with GT Label
        axes[0].imshow(img)
        axes[0].set_title(f"Original Colonoscopy\n(Mayo Grade: {row['mayo_label']})",
                          fontsize=10, fontweight='bold')
        axes[0].axis('off')

        # 2. Right: Segmentation Overlay
        # We load the mask and overlay it on the original for better context
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        axes[1].imshow(img)
        overlay = np.zeros_like(img)
        overlay[mask > 0] = [0, 115, 207] # Medical Blue
        axes[1].imshow(overlay, alpha=0.5)
        axes[1].set_title("AI-Driven Mucosal Segmentation", fontsize=10, fontweight='bold')
        axes[1].axis('off')

        # 3. FIX: Explicitly target the clinical_caption column
        # We also manually prepend 'Colonoscopy Findings' to anchor the context
        raw_caption = str(row['clinical_caption'])

        # Ensure coordinates aren't being printed as the caption
        if raw_caption.startswith("[ymin"):
            caption_to_show = "⚠️ ERROR: Coordinate/Caption Swap detected in CSV."
        else:
            caption_to_show = f"Colonoscopy Analysis:\n{raw_caption}"

        wrapped_caption = "\n".join(textwrap.wrap(caption_to_show, width=85))

        plt.suptitle(wrapped_caption, y=0.1, fontsize=9, color='blue',
                     fontweight='bold', ha='center', va='top')

        plt.tight_layout(rect=[0, 0.15, 1, 0.95])
        plt.show()

# Run the evaluation on your generated dataset
run_final_presentation_eval("/content/UC_Enhanced_Dataset/enhanced_metadata.csv")

# **Finalized Data Characteristics**

### **Labels and Caption Length Distrubition**

In [17]:
import pandas as pd

# Load your csv
df = pd.read_csv('/content/enhanced_7B_descriptive.csv')

# 1. Calculate Class Counts and Ratios
class_counts = df['original_mayo'].value_counts()
class_ratios = df['original_mayo'].value_counts(normalize=True) * 100

# 2. Calculate Average Caption Length per Class
# This helps see if one class has more "information" than others
avg_caption_len = df.groupby('original_mayo')['caption_length'].mean()

# 3. Create a Summary Table
imbalance_report = pd.DataFrame({
    'Total Samples': class_counts,
    'Percentage (%)': class_ratios.map('{:.2f}%'.format),
    'Avg Caption Length': avg_caption_len.map('{:.1f} chars'.format)
}).sort_index()

print("--- Class Imbalance Report ---")
print(imbalance_report)

# 4. Check for Extreme Imbalance
max_ratio = class_ratios.max()
min_ratio = class_ratios.min()
if max_ratio / min_ratio > 3:
    print("\n[WARNING]: High imbalance detected.")
else:
    print("\n[INFO]: Dataset is relatively balanced.")

--- Class Imbalance Report ---
               Total Samples Percentage (%) Avg Caption Length
original_mayo                                                 
Mayo 0                  3668         54.46%        489.0 chars
Mayo 1                  1826         27.11%        498.7 chars
Mayo 2                   710         10.54%        457.9 chars
Mayo 3                   531          7.88%        518.3 chars

[WARNING]: High imbalance detected.


### **Mask/Aspect Ratio Check**

In [7]:
import pandas as pd
import os
from tqdm import tqdm

# 1. Load the CSV
csv_path = '/content/enhanced_7B_descriptive.csv'
df = pd.read_csv(csv_path)
initial_count = len(df)

print(f"🔍 Verifying and cleaning {initial_count} rows...")

# 2. Identify rows where BOTH files exist
valid_indices = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Checking Disk"):
    img_p = str(row['original_image'])
    mask_p = str(row['mask_image'])

    # Only keep the index if both files are physically present
    if os.path.exists(img_p) and os.path.exists(mask_p):
        valid_indices.append(index)

# 3. Create the cleaned DataFrame
df_cleaned = df.loc[valid_indices].reset_index(drop=True)
final_count = len(df_cleaned)
removed_count = initial_count - final_count

# 4. SAVE the cleaned CSV
# This overwrites the old file so your training loop uses the correct data
df_cleaned.to_csv(csv_path, index=False)

# --- SUMMARY REPORT ---
print("\n" + "="*35)
print("📊 CLEANING & SAVING COMPLETE")
print("="*35)
print(f"Initial Rows:     {initial_count}")
print(f"Rows Removed:     {removed_count}")
print(f"Final Valid Rows: {final_count}")
print(f"File Saved To:    {csv_path}")
print("="*35)

if final_count == 0:
    print("\n🚨 CRITICAL WARNING: No images were found. Do not start training yet!")
else:
    print("\n✅ Your dataset is now perfectly synced with your local disk.")

🔍 Verifying and cleaning 6735 rows...


Checking Disk: 100%|██████████| 6735/6735 [00:00<00:00, 19014.07it/s]



📊 CLEANING & SAVING COMPLETE
Initial Rows:     6735
Rows Removed:     340
Final Valid Rows: 6395
File Saved To:    /content/enhanced_7B_descriptive.csv

✅ Your dataset is now perfectly synced with your local disk.


In [28]:
import pandas as pd
import numpy as np
from PIL import Image
import os
from tqdm import tqdm

# Load CSV once for all cells
df = pd.read_csv('/content/enhanced_7B_descriptive.csv')

print("[1/4] Checking Mask Validity and Aspect Ratios...")
empty_masks = []
aspect_ratios = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        # Check for empty masks
        mask = Image.open(row['mask_image']).convert('L')
        if np.array(mask).sum() == 0:
            empty_masks.append(row['mask_image'])

        # Check aspect ratio
        img = Image.open(row['original_image'])
        w, h = img.size
        aspect_ratios.append(w / h)
    except Exception as e:
        print(f"Error loading {idx}: {e}")

print(f"\n✅ Analysis Complete.")
print(f"❌ Found {len(empty_masks)} empty masks (all black pixels).")
print(f"📊 Average Aspect Ratio: {np.mean(aspect_ratios):.2f}")
if len(empty_masks) > 0:
    print("👉 Recommendation: Remove rows with empty masks before training.")

[1/4] Checking Mask Validity and Aspect Ratios...


100%|██████████| 6395/6395 [00:05<00:00, 1150.73it/s]


✅ Analysis Complete.
❌ Found 0 empty masks (all black pixels).
📊 Average Aspect Ratio: 1.22


### **Patient Distribution**

In [25]:
print("Analyzing Patient Distribution...")

patient_counts = df['patient_id'].value_counts()
total_images = len(df)

print(f"Total Unique Patients: {len(patient_counts)}")
print(f"Average images per patient: {patient_counts.mean():.1f}")
print(f"Top Patient Contribution: {(patient_counts.max()/total_images)*100:.2f}% of total data")

if (patient_counts.max() / total_images) > 0.15:
    print("⚠️ Warning: One patient represents over 15% of your data. Consider downsampling them.")
else:
    print("✅ Patient distribution looks healthy.")

Analyzing Patient Distribution...
Total Unique Patients: 312
Average images per patient: 20.5
Top Patient Contribution: 1.64% of total data
✅ Patient distribution looks healthy.


# **Classfier Implementation**

### **Dataset Class (Handling the Folder Nesting)**

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class HuluMedDatasetQwen(Dataset):
    def __init__(self, df, img_size=224):
        self.df = df.reset_index(drop=True)
        self.processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-8B-Instruct")

        self.img_transform = T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor(),
        ])
        self.mask_transform = T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor()
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img = Image.open(row['original_image']).convert('RGB')
        mask = Image.open(row['mask_image']).convert('L')

        img_t = self.img_transform(img)
        mask_t = self.mask_transform(mask)

        alpha = 0.3
        masked_img_t = (img_t * mask_t) + (img_t * (1 - mask_t) * alpha)
        masked_img_pil = T.ToPILImage()(masked_img_t)

        # FIX: Prepend vision placeholder tokens so the model knows where to put image features
        prompt = f"<|vision_start|><|image_pad|><|vision_end|>{str(row['clinical_caption'])}"

        inputs = self.processor(
            text=[prompt],
            images=[masked_img_pil],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512
        )

        label = int(str(row['original_mayo']).split(' ')[-1])

        return {
            "pixel_values": inputs['pixel_values'].squeeze(0),
            "input_ids": inputs['input_ids'].squeeze(0),
            "attention_mask": inputs['attention_mask'].squeeze(0),
            "image_grid_thw": inputs['image_grid_thw'].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

### **Model Architecture**

In [2]:
class QwenClassifier(nn.Module):
    def __init__(self, vlm, num_classes=4):
        super().__init__()
        self.vlm = vlm
        # Use text_config for the hidden size
        hidden_size = vlm.config.text_config.hidden_size
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, input_ids, attention_mask, pixel_values, image_grid_thw):
        outputs = self.vlm(
            input_ids=input_ids,
            attention_mask=attention_mask,
            pixel_values=pixel_values,
            image_grid_thw=image_grid_thw,
            output_hidden_states=True, # Required to get the hidden_states tuple
            return_dict=True
        )

        # Qwen3VLCausalLMOutputWithPast stores hidden states in a tuple
        # The last hidden state is the last element of that tuple
        # Shape: (batch_size, sequence_length, hidden_size)
        last_hidden_state = outputs.hidden_states[-1]

        # Average pooling across the sequence dimension
        pooled = last_hidden_state.mean(dim=1)

        return self.classifier(pooled)
# --- Collation Function ---
def qwen_collate_fn(batch):
    return {
        "pixel_values": torch.stack([item['pixel_values'] for item in batch]),
        "input_ids": torch.stack([item['input_ids'] for item in batch]),
        "attention_mask": torch.stack([item['attention_mask'] for item in batch]),
        "image_grid_thw": torch.stack([item['image_grid_thw'] for item in batch]),
        "label": torch.stack([item['label'] for item in batch]),
    }

In [3]:
import bitsandbytes as bnb
from transformers import AutoProcessor, AutoModelForVision2Seq, BitsAndBytesConfig # Corrected import
import torch # Added missing torch import

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Processor
processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-8B-Instruct")

# 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load Qwen3-VL using the repo's custom model class
vlm_model = AutoModelForVision2Seq.from_pretrained( # Corrected class
    "Qwen/Qwen3-VL-8B-Instruct",
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config
)
vlm_model.gradient_checkpointing_enable()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### **Patient-Wise Splitting Logic**

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/enhanced_7B_descriptive.csv')

unique_patients = df['patient_id'].unique()
train_pts, val_pts = train_test_split(unique_patients, test_size=0.2, random_state=42)

train_df = df[df['patient_id'].isin(train_pts)].reset_index(drop=True)
val_df = df[df['patient_id'].isin(val_pts)].reset_index(drop=True)

print(f"Training on {len(train_df)} images from {len(train_pts)} patients.")
print(f"Validating on {len(val_df)} images from {len(val_pts)} patients.")


Training on 5136 images from 249 patients.
Validating on 1259 images from 63 patients.


### **The Training and Validation Loop**

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from torch.amp import autocast
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm

# --- CONFIGURATION FOR A100 SPEED ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16          # A100 can easily handle 16-32 with Qwen3-VL 8B (8-bit)
ACCUM_STEPS = 2         # Effective Batch Size = 32
LR = 2e-5
NUM_EPOCHS = 7
AMP_DTYPE = torch.bfloat16 # Native to A100, faster and avoids 'unscale' errors

# 1. Compute Class Weights
y_train = train_df['original_mayo'].apply(lambda x: int(str(x).split(' ')[-1])).values
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)

# 2. Optimized Dataloaders for A100
train_loader = DataLoader(
    HuluMedDatasetQwen(train_df),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=8,           # Use more cores for image decoding
    pin_memory=True,         # Essential for fast data transfer to GPU
    persistent_workers=True, # Keeps workers alive between epochs
    collate_fn=qwen_collate_fn
)

val_loader = DataLoader(
    HuluMedDatasetQwen(val_df),
    batch_size=BATCH_SIZE * 2, # Evaluation doesn't need grads; use double batch
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,
    collate_fn=qwen_collate_fn
)

# 3. Model Setup
model = QwenClassifier(vlm_model, num_classes=4).to(DEVICE)

# Note: In 8-bit mode, the optimizer handles internal scaling
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=0.01)
criterion = CrossEntropyLoss(weight=class_weights)

best_qwk = -1.0

# --- TRAINING LOOP ---
for epoch in range(NUM_EPOCHS):
    vlm_model.train()
    model.classifier.train()
    train_loss = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")

    optimizer.zero_grad(set_to_none=True) # Slightly faster than zero_grad()

    for i, batch in enumerate(pbar):
        # Move batch to device (non_blocking=True improves overlap)
        pixel_values = batch['pixel_values'].to(DEVICE, non_blocking=True)
        input_ids = batch['input_ids'].to(DEVICE, non_blocking=True)
        attention_mask = batch['attention_mask'].to(DEVICE, non_blocking=True)
        image_grid_thw = batch['image_grid_thw'].to(DEVICE, non_blocking=True)
        labels = batch['label'].to(DEVICE, non_blocking=True)

        # A100 Optimized Mixed Precision
        with autocast(device_type="cuda", dtype=AMP_DTYPE):
            logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                pixel_values=pixel_values,
                image_grid_thw=image_grid_thw
            )
            loss = criterion(logits, labels) / ACCUM_STEPS

        # Quantized training handles its own scaling; GradScaler is NOT needed
        loss.backward()

        if (i + 1) % ACCUM_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

        train_loss += loss.item() * ACCUM_STEPS
        pbar.set_postfix({'loss': f"{(loss.item()*ACCUM_STEPS):.4f}"})

    # --- EVALUATION ---
    vlm_model.eval()
    model.classifier.eval()
    val_preds, val_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            pixel_values = batch['pixel_values'].to(DEVICE)
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            image_grid_thw = batch['image_grid_thw'].to(DEVICE)
            labels = batch['label'].to(DEVICE)

            with autocast(device_type="cuda", dtype=AMP_DTYPE):
                logits = model(input_ids, attention_mask, pixel_values, image_grid_thw)

            preds = torch.argmax(logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    acc = np.mean(np.array(val_preds) == np.array(val_labels))
    qwk = cohen_kappa_score(val_labels, val_preds, weights='quadratic')

    print(f"Epoch {epoch+1} | Loss: {train_loss/len(train_loader):.4f} | Acc: {acc:.4f} | QWK: {qwk:.4f}")

    if qwk > best_qwk:
        best_qwk = qwk
        torch.save({
            "vlm_model": vlm_model.state_dict(),
            "classifier": model.classifier.state_dict()
        }, 'best_qwen_mayo_qwk.pth')
        print(f"⭐ New Best QWK: {best_qwk:.4f}. Model Saved!")

Epoch 1/7:   0%|          | 0/321 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Epoch 1/7:   0%|          | 1/321 [00:48<4:19:39, 48.68s/it, loss=3.5106]

### **Test Set Loader**

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Load the Test Dataset
test_df = pd.read_csv('your_test_dataset.csv')
test_dataset = ModelCDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 2. Load your BEST saved model
model = MedCLIPClassifier(num_classes=4).to(device)
model.load_state_dict(torch.load('best_medclip_mayo.pth'))
model.eval()

all_preds = []
all_labels = []

# 3. Run Inference
print("Evaluating on Test Set...")
with torch.no_grad():
    for batch in tqdm(test_loader):
        pixel_values = batch['pixel_values'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        logits = model(pixel_values, input_ids, attention_mask)
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

### **Evaluation**



1.   Classification Report




In [ ]:
# --- METRIC 1: Classification Report (Precision, Recall, F1) ---
report = classification_report(all_labels, all_preds,
                               target_names=['Mayo 0', 'Mayo 1', 'Mayo 2', 'Mayo 3'])
print("\n--- Detailed Classification Report ---")
print(report)

# --- METRIC 2: Confusion Matrix Visual ---
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Mayo 0', 'Mayo 1', 'Mayo 2', 'Mayo 3'],
            yticklabels=['Mayo 0', 'Mayo 1', 'Mayo 2', 'Mayo 3'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix: Mayo Clinic Score Prediction')
plt.show()

# --- METRIC 3: Weighted F1 Score (Best for Imbalance) ---
final_f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"\nFinal Weighted F1-Score: {final_f1:.4f}")




2.   Quadratic Weighted Kappa (QWK)



In [ ]:
from sklearn.metrics import cohen_kappa_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def evaluate_classification(all_labels, all_preds):
    # QWK penalizes a "Mayo 0 vs 3" error more than a "Mayo 0 vs 1" error
    qwk = cohen_kappa_score(all_labels, all_preds, weights='quadratic')

    # Simple Accuracy for reference
    acc = sum(1 for p, l in zip(all_preds, all_labels) if p == l) / len(all_labels)

    print(f"Quadratic Weighted Kappa: {qwk:.4f}")
    print(f"Overall Accuracy: {acc:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=['Mayo 0', 'Mayo 1', 'Mayo 2', 'Mayo 3'])
    disp.plot(cmap='Blues')
    plt.title("Mayo Score Classification Results")
    plt.show()